In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import cPickle as pickle
import sys, os, time, csv, random
from pandas import DataFrame as df
from numpy import linalg as LA
import matplotlib.pyplot as plt
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, TimeDistributedDense, Masking
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam, RMSprop, SGD, Adagrad, Adamax


Using Theano backend.


### Load data from pickle file and prepare training set and testing set

In [2]:
DATA_PATH = '/Users/hi08060204/Desktop/Peter_ML/MIMIC_Project/Data/Processed/mimic3_11_resampled-imputed_5000.npz'

nb_samples = None
test_split = 0.1

# Loading data from .npz file into dictionary form
data = np.load(DATA_PATH)
X_raw = data['X'][0:nb_samples]
y_raw = data['ylos'][0:nb_samples]

# Spliting data for the use of training and testing
X_raw_test = X_raw[0:len(X_raw)*test_split]
y_raw_test = y_raw[0:len(y_raw)*test_split]
X_raw_train = X_raw[len(X_raw)*test_split:]
y_raw_train = y_raw[len(y_raw)*test_split:]

/Users/hi08060204/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/hi08060204/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/hi08060204/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/hi08060204/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### Prepare sequential output, specifying remaining hours

In [3]:
X_train = X_raw_train
y_train_list = [ [ [left_hours] for left_hours in reversed(range(ep.shape[0]))] for ep in X_train ] 
y_train = np.array(y_train_list)

X_test = X_raw_test
y_test_list = [ [ [left_hours] for left_hours in reversed(range(ep.shape[0]))] for ep in X_test ]
y_test = np.array(y_test_list)

## Create a dataframe for batch manipulation

In [4]:
# Create training dataframe
pred_hours = 8
# hours_left = []
# dis_list = []
new_epiosdes = []
count_down = []
y_out = []

for x, y in zip(X_raw_train, y_train_list):
    dis = np.random.binomial(1, 0.5)
    
    if dis==1:
        rand_cut = random.randrange(0, -pred_hours, -1)
#         hours_left.append(-rand_cut)
#         dis_list.append(dis)
        y_out.append([dis, -rand_cut])
        
        if rand_cut==0:
            new_epiosdes.append(x[:None]) 
            count_down.append(y[:None])
        else:
            new_epiosdes.append(x[:rand_cut])
            count_down.append(y[:rand_cut])
        
    else:

        rand_cut = random.randrange(-pred_hours, -x.shape[0], -1)
#         hours_left.append(-rand_cut)
#         dis_list.append(dis)
        y_out.append([dis, -rand_cut])

        new_epiosdes.append(x[:rand_cut])
        count_down.append(y[:rand_cut])


train_df = df({'episodes': new_epiosdes, 
               'ori_hours' : [h.shape[0] for h in X_raw_train],
               'hours_pass': [h.shape[0] for h in new_epiosdes],
               'ylos': y_raw_train,
               'count_down': count_down,
#                'hours_left': hours_left,
#                'dis': dis_list,
               'dis_hrLeft': y_out}) #.sort_values(by ='hours')


train_df

,count_down,dis_hrLeft,episodes,hours_pass,ori_hours,ylos
0,[[152]],"[0, 152]","[[51.0, 129.0, 0.0, 0.21, 15.0, 128.0, 76.0, 7...",1,153,6.3564
1,[[21]],"[0, 21]","[[70.0, 120.666666667, 0.0, 0.21, 14.0, 168.0,...",1,22,0.9061
2,[[25]],"[0, 25]","[[59.0, 118.0, 0.0, 0.21, 11.0, 148.0, 86.0, 7...",1,26,1.1295
3,"[[12], [11], [10], [9], [8]]","[0, 8]","[[62.0, 119.0, 0.0, 0.21, 15.0, 128.0, 79.5, 7...",5,13,0.5208
4,"[[121], [120], [119], [118], [117], [116], [11...","[0, 97]","[[74.6666666667, 135.333333333, 0.0, 0.21, 15....",25,122,5.0933
5,"[[66], [65], [64], [63], [62], [61], [60], [59...","[1, 7]","[[108.666666667, 213.666666667, 0.0, 0.21, 15....",60,67,2.9585
6,"[[210], [209], [208], [207], [206], [205], [20...","[0, 49]","[[111.0, 154.0, 0.0, 0.21, 11.0, 98.0, 134.666...",162,211,8.9064
7,"[[22], [21], [20], [19], [18], [17], [16], [15...","[1, 6]","[[97.0, 140.0, 0.0, 0.21, 11.0, 114.0, 97.6666...",17,23,0.9519
8,"[[56], [55], [54], [53], [52], [51], [50], [49...","[1, 3]","[[74.0, 121.5, 0.0, 0.21, 11.0, 66.0, 99.66666...",54,57,2.3906
9,"[[23], [22], [21], [20], [19]]","[0, 19]","[[69.0, 114.0, 0.0, 0.21, 3.0, 170.0, 95.0, 7....",5,24,1.0641


In [5]:
# pred_hours = 8
# hours_left = []
# dis_list = []
new_epiosdes = []
count_down = []
y_out = []

for x, y in zip(X_raw_test, y_test_list):
    dis = np.random.binomial(1, 0.5)
    
    if dis==1:
        rand_cut = random.randrange(0, -pred_hours, -1)
#         hours_left.append(-rand_cut)
#         dis_list.append(dis)
        y_out.append([dis, -rand_cut])

        
        if rand_cut==0:
            new_epiosdes.append(x[:None]) 
            count_down.append(y[:None])
        else:
            new_epiosdes.append(x[:rand_cut])
            count_down.append(y[:rand_cut])
        
    else:
        rand_cut = random.randrange(-pred_hours, -x.shape[0], -1)
#         hours_left.append(-rand_cut)
#         dis_list.append(dis)
        y_out.append([dis, -rand_cut])

        new_epiosdes.append(x[:rand_cut])
        count_down.append(y[:rand_cut])
        
test_df = df({'episodes': new_epiosdes, 
               'ori_hours' : [h.shape[0] for h in X_raw_test],
               'hours_pass': [h.shape[0] for h in new_epiosdes],
               'ylos': y_raw_test,
               'count_down': count_down,
#                'hours_left': hours_left,
#                'dis': dis_list,
               'dis_hrLeft': y_out}) #.sort_values(by ='hours')


test_df

,count_down,dis_hrLeft,episodes,hours_pass,ori_hours,ylos
0,"[[48], [47], [46], [45], [44], [43], [42], [41...","[0, 14]","[[59.0, 118.0, 0.0, 0.21, 11.0, 144.0, 86.0, 7...",35,49,2.1167
1,"[[871], [870], [869], [868], [867], [866], [86...","[1, 3]","[[45.0, 100.0, 0.0, 0.21, 15.0, 128.0, 83.0, 7...",869,872,36.7002
2,"[[31], [30], [29], [28], [27], [26], [25], [24...","[0, 21]","[[74.0, 130.0, 0.0, 0.21, 15.0, 75.0, 57.0, 7....",11,32,1.3263
3,"[[35], [34], [33], [32], [31], [30], [29], [28...","[1, 2]","[[85.5, 139.5, 0.0, 0.21, 7.0, 142.0, 91.0, 7....",34,36,1.4951
4,"[[250], [249], [248], [247], [246], [245], [24...","[0, 28]","[[37.0, 118.0, 0.0, 0.21, 7.0, 304.0, 88.0, 7....",223,251,10.4665
5,"[[14], [13], [12]]","[0, 12]","[[67.0, 131.0, 0.0, 0.21, 11.0, 128.0, 92.0, 7...",3,15,0.7325
6,"[[34], [33], [32], [31], [30], [29]]","[0, 29]","[[50.0, 91.0, 0.0, 0.21, 15.0, 128.0, 101.5, 7...",6,35,1.6325
7,"[[185], [184], [183], [182], [181], [180], [17...","[0, 47]","[[59.0, 118.0, 0.0, 0.21, 11.0, 203.5, 86.0, 7...",139,186,8.2914
8,"[[21], [20], [19], [18], [17], [16], [15], [14...","[0, 12]","[[60.0, 152.0, 0.0, 0.21, 14.0, 128.0, 85.0, 7...",10,22,0.8957
9,"[[39], [38], [37], [36], [35], [34], [33], [32...","[0, 14]","[[52.0, 112.0, 0.0, 0.21, 15.0, 128.0, 77.3333...",26,40,1.7126


#### Group batches that have same time length

In [6]:
# Create a model with input = (nb_samples, tsteps, nb_features); output = (nb_samples, 1)
print('Creating Model...')
nb_features = 14
last_layer_nodes = 1024

buildStart_time = time.time()
model = Sequential()
model.add(LSTM(output_dim = 64, 
               return_sequences = True, 
               input_dim = nb_features,
               # input_shape = (tsteps, nb_features), 
               # dropout_U=0.5, 
               activation='sigmoid', 
               inner_activation='hard_sigmoid'))

# model.add(TimeDistributedDense(output_dim = 64, activation="sigmoid"))

model.add(LSTM(output_dim = last_layer_nodes, 
               # return_sequences = True,
               # input_dim = nb_features,
               # input_shape = (tsteps, nb_features), 
               # dropout_U=0.5,
               activation='sigmoid',
               inner_activation='hard_sigmoid'))

# model.add(TimeDistributedDense(output_dim = 1, activation="linear"))

# model.add(Dense(output_dim = 32,
#                 activation = 'linear'))

model.add(Dense(output_dim = 1,
                activation = 'sigmoid'))

buildEnd_time = time.time()


# Compile the model
print("Build model time:", buildEnd_time - buildStart_time, "secs")
highLearn_adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='mse', optimizer= highLearn_adam) #, sample_weight_mode = "temporal")

compileEnd_time = time.time()
print("Compile time:", compileEnd_time-buildEnd_time, "secs")

Creating Model...
Build model time: 10.0223309994 secs
Compile time: 109.663897991 secs


In [7]:
train_df_gb = train_df.groupby('hours_pass')
nb_epochs = 200

cutBatch = 90000
cutTimeLen = 100
loss_trend = []
acc_trend = []
uni_train_hours = np.unique(np.asarray(train_df['hours_pass']))
np.random.shuffle(uni_train_hours)

trainStart_time = time.time()
temp = trainStart_time

# Number of times of iteration on the "WHOLE" data set
print("Training Model...")
for epoch_it in range(nb_epochs):
    inEpo_loss_trend = []
    inEpo_acc_trend = []
    batchCount = 0
    np.random.shuffle(uni_train_hours)
    
    # Grouping and training on batches of samples according to the length of time
    for time_len in uni_train_hours:
        
        if batchCount > cutBatch or time_len > cutTimeLen:
            continue
            
        else:
            maxTrainLen = time_len
            lastTrainBat = batchCount
            
            g = train_df_gb.get_group(time_len)

            batch_size = len( g['episodes'])
            X_train = np.dstack(np.asarray(g['episodes']))
            X_train = np.transpose(X_train, (2,0,1))
            
            y_train = [ d_h[0] for d_h in g['dis_hrLeft']]



            trainMid_time = time.time()
            
#             # Create output mask
#             out_mask_step = 10
#             output_mask = np.zeros(shape = (batch_size, time_len))    
#             ## Specify timesteps to be revealed
#             x = np.ones(shape = (batch_size, output_mask[:,::-10].shape[1] ) )
#             output_mask[:,::-out_mask_step] = x
            
            (loss, acc) = model.train_on_batch(X_train, 
                                               y_train,
                                               accuracy = True) #, sample_weight = output_mask)
        
        
            p = ("Epoch:" 
                 + str(epoch_it+1)
                 + "/"
                 + str(nb_epochs)
                 + " Time length: " 
                 + str(time_len) 
                 + ", # of samples: " 
                 + str(batch_size)
                 + " || loss = "
                 + str(round(loss,4))
                 + ", acc = "
                 + str(round(acc,4))
                 + ". It's been "
                 + str(round(trainMid_time-trainStart_time, 1))
                 + ' secs')
            sys.stdout.write('\r' + p)

            inEpo_loss_trend.append(float(loss))
            inEpo_acc_trend.append(float(acc))
            batchCount = batchCount + 1
    
    
    aEpoch_time = time.time()
    epochLapse = aEpoch_time - temp
    inEpo_avg_loss = sum(inEpo_loss_trend)/len(inEpo_loss_trend)
    inEpo_avg_acc = sum(inEpo_acc_trend)/len(inEpo_acc_trend)
    
    loss_trend.append(inEpo_avg_loss)
    acc_trend.append(inEpo_avg_acc)
    avg_loss = sum(loss_trend)/len(loss_trend)
    avg_acc = sum(acc_trend)/len(acc_trend)
    
    print('\nThis epoch takes:', round(epochLapse,4),
          "secs, loss:", inEpo_avg_loss,
          "acc:", inEpo_avg_acc,
          "|| (Overall) Avg. Loss:", round(avg_loss,4), 
          "Avg. Acc:", round(avg_acc,4),
          "\n")
    
    temp = aEpoch_time

trainEnd_time = time.time()
print ("\nDone!\nThis takes:", trainEnd_time-trainStart_time, "seconds")

Training Model...
Epoch:1/200 Time length: 42, # of samples: 51 || loss = 0.2952, acc = 0.2549. It's been 399.0 secs
This epoch takes: 403.7179 secs. Avg. Loss: 0.3058 Avg. Acc: 0.5226 

Epoch:2/200 Time length: 86, # of samples: 20 || loss = 0.2137, acc = 0.7. It's been 804.8 secs
This epoch takes: 406.4819 secs. Avg. Loss: 0.2588 Avg. Acc: 0.5497 

Epoch:3/200 Time length: 57, # of samples: 22 || loss = 0.257, acc = 0.4091. It's been 1210.7 secs
This epoch takes: 404.0401 secs. Avg. Loss: 0.2539 Avg. Acc: 0.5413 

Epoch:4/200 Time length: 23, # of samples: 116 || loss = 0.2421, acc = 0.6379. It's been 1616.1 secs
This epoch takes: 406.8731 secs. Avg. Loss: 0.2481 Avg. Acc: 0.5671 

Epoch:5/200 Time length: 94, # of samples: 19 || loss = 0.2355, acc = 0.6842. It's been 2030.4 secs
This epoch takes: 414.5397 secs. Avg. Loss: 0.2415 Avg. Acc: 0.5888 

Epoch:6/200 Time length: 10, # of samples: 91 || loss = 0.2008, acc = 0.8791. It's been 2435.7 secs
This epoch takes: 401.733 secs. Avg. 

In [14]:
# Test all the output

y_test = [ d_h[0] for d_h in test_df['dis_hrLeft']]
y_predict = []
print("Now Predicting...")

for i, X_test in enumerate(test_df['episodes']): 
    
    X_test = X_test[None,:,:] 
    y_result = model.predict(X_test)
    y_predict.append(float(y_result))
    per = round( ((i+1)*100)/test_df.shape[0] , 4)
    p = (str(i+1)
         + "/"
         + str(len(test_df['episodes']))
         + " predicted. "
         + str(per)
         + "%" )
         
    sys.stdout.write('\r' + p)

# compare = df({'y_predict': y_predict, 'y_test': y_test})  

# y_predict = np.asarray(compare.y_predict)
# y_test = np.asarray(compare.y_test)

y_predict = np.array(y_predict)
y_test = np.array(y_test)

testLoss = abs(y_predict-y_test)
test_acc = sum(testLoss<0.5)/len(y_test)
mseTestLoss = LA.norm(testLoss)
# avgMseTest = sum(mseTestLoss)/len(mseTestLoss)
# print(mseTestLoss)
 
# np.unique(compare.y_predict)

Now Predicting...
678/678 predicted. 100.0%

In [28]:
test_acc = sum(testLoss<0.5)/float(y_predict.shape[0])

print(test_acc)

0.685840707965


In [33]:
# model.reset_states()
test_index = 250
X_test = test_df.episodes.iloc[test_index][None,:,:]

y_result = model.predict(X_test)

y_test = test_df.dis_hrLeft.iloc[test_index]
print(y_test, y_result)


[0, 38] [[ 0.00887627]]


In [20]:
SAVE_PATH = "/Users/hi08060204/Desktop/Peter_ML/MIMIC_Project/Result"
loss_filename = "loss_trend_dis_" + str(nb_epochs) + "Epo.png"
plt.figure()
plt.plot(loss_trend)
plt.savefig(os.path.join(SAVE_PATH, loss_filename))

acc_filename = "acc_trend_dis_" + str(nb_epochs) + "Epo.png"
plt.figure()
plt.plot(acc_trend)
plt.savefig(os.path.join(SAVE_PATH, acc_filename))

testLoss_filename = "testLoss_dis_" + str(nb_epochs) + "Epo.png"
plt.figure()
plt.plot(testLoss)
plt.savefig(os.path.join(SAVE_PATH, testLoss_filename))

In [34]:
loss_filename = "loss_trend_dis_" + str(nb_epochs) + "Epo.p"
pickle.dump( loss_trend, open( os.path.join(SAVE_PATH, loss_filename), "wb") )

acc_filename = "acc_trend_dis_" + str(nb_epochs) + "Epo.p"
pickle.dump( acc_trend, open( os.path.join(SAVE_PATH, acc_filename), "wb") )

testLoss_filename = "testLoss_dis_" + str(nb_epochs) + "Epo.p"
pickle.dump( testLoss, open( os.path.join(SAVE_PATH, testLoss_filename), "wb") )

In [19]:
# avgMseTest = sum(mseTestLoss)/len(mseTestLoss)
mseTestLoss

11.796681767432244

In [36]:

model_weight = model.get_weights()

modelWeight_filename = "modelWeight_dis_" + str(nb_epochs) + "Epo"
pickle.dump( model_weight, open( os.path.join(SAVE_PATH, modelWeight_filename), "wb") )

model.save_weights( os.path.join(SAVE_PATH, modelWeight_filename) )

json_string = model.to_json()


[WARNING] /Users/hi08060204/Desktop/Peter_ML/MIMIC_Project/Result/modelWeight_dis_200Epo already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True in save_weights!


In [40]:
json_string = model.to_json()

json_fn = "json_dis_" + str(nb_epochs) + "Epo.txt"
# json_f = open( os.path.join(SAVE_PATH, json_fn), "wb") 
with open( os.path.join(SAVE_PATH, json_fn), "wb") as json_f:
    json_f.write(json_string)
    
    